In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch


In [2]:
data = pd.read_csv("clean_weather.csv")
data = data.ffill()


In [4]:
DEVICE=  torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from torch.utils.data import DataLoader,Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.data) - self.sequence_length
        
    def __getitem__(self, idx):
        # Get sequence of data
        sequence = self.data[idx:idx + self.sequence_length]
        return sequence

def transform(data,batch_size,sequence_len,device,dtype=torch.float32):
    
    data_tensor = torch.tensor(data,dtype=dtype,device=device)
    dataset = TimeSeriesDataset(data_tensor,sequence_len)
    
    dataloader = DataLoader(dataset,batch_size,shuffle=False)
    
    return dataloader
    

In [9]:
# Assuming 'date' is your date column and data is sorted by date
from sklearn.preprocessing import StandardScaler

predictors = ["tmax", "tmin", "rain"]
target = "tmax_tomorrow"

# Calculate split points
total_rows = len(data)
train_end = int(0.7 * total_rows)
valid_end = int(0.85 * total_rows)

# Split the data
X_train = data[predictors].iloc[:train_end]
y_train = data[target].iloc[:train_end]

X_valid = data[predictors].iloc[train_end:valid_end]
y_valid = data[target].iloc[train_end:valid_end]

X_test = data[predictors].iloc[valid_end:]
y_test = data[target].iloc[valid_end:]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Convert to a dataloader object with batch_size

X_train_scaled = np.array(X_train_scaled)
X_valid_scaled = np.array(X_valid_scaled)
X_test_scaled = np.array(X_test_scaled)

#We dont scale target data check on google for more info on why
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

In [10]:
x_train = transform(X_train_scaled,32,7,DEVICE)
x_valid = transform(X_valid_scaled,32,7,DEVICE)
x_test = transform(X_test_scaled,32,7,DEVICE)
y_train = transform(y_train,32,7,DEVICE)
y_valid = transform(y_valid,32,7,DEVICE)
y_test = transform(y_test,32,7,DEVICE)

In [21]:
import torch.nn as nn
import torch.optim as optim
import math
from tqdm import tqdm

class SimpleRNN(nn.Module):
    def __init__(self, input_size,hidden_size,output_size):
        super(SimpleRNN,self).__init__()
        
        torch.manual_seed(0)
        
        k = 1/math.sqrt(hidden_size)
        
        #input to hidden layer
        self.i2h = nn.Linear(input_size,hidden_size)
        self.i2h.weight.data.uniform_(-k,k)
        self.i2h.bias.data.uniform_(-k,k)
        
        #hidden to hidden layer
        self.h2h = nn.Linear(hidden_size,hidden_size)
        self.h2h.weight.data.uniform_(-k,-k)
        self.h2h.bias.data.uniform_(-k,k)
        
        #hidden to ouput layer
        self.h2o = nn.Linear(hidden_size,output_size)
        self.h2o.weight.data.uniform_(-k,k)
        self.h2o.bias.data.uniform_(-k,k)
        
        #tanh
        self.tanh = nn.Tanh()
        self.hidden_size = hidden_size
        
    def init_hidden(self,batch_size,device=torch.device('cuda')):
        return torch.zeros(batch_size,self.hidden_size,device=device)
        
    def forward(self,x,hidden):
        batch_size = x.shape[0]
        seq_len = x.shape[1]
        
        if hidden is None:
            hidden = self.init_hidden(batch_size,device=x.device)
        
        outputs = []
        
        for t in range(seq_len):
            #get current input
            x_t = x[:,t,:]
        
            #combine input and hidden state and apply activation
            input_t = self.i2h(x_t)
            hidden_t = self.h2h(hidden)
            hidden = self.tanh(input_t+hidden_t)
            
            #output layer
            output = self.h2o(hidden)
            outputs.append(output)
            
        #stack outputs along sequence dimensions
        outputs = torch.stack(outputs,dim=1)
        return outputs, hidden
    
    
        

In [18]:
def train_model(model,trainx_loader,trainy_loader,validx_loader,validy_loader,num_epochs,learning_rate):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)
        
        #mean squared error
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(),lr=learning_rate)
        
        #to store metrics
        train_losses = []
        val_losses = []
        
        for epoch in range(num_epochs):
            model.train()
            total_train_loss = 0
            hidden = None
            
            #Training loop 
            for batch_idx, (x_batch,y_batch) in enumerate(tqdm(zip(trainx_loader,trainy_loader))):
                x_batch = x_batch.to(device) #shape (32,7,3)
                y_batch = y_batch.to(device) #shape(32,7)
                
                #forward pass 
                outputs, hidden = model(x_batch,hidden) #outputs shape(32, 7, 1)
                outputs = outputs.squeeze(-1) #shape (32,7)
                
                #calculate loss
                loss = criterion(outputs,y_batch)
                
                #backward pass and optimize
                optimizer.zero_grad()
                loss.backward()
                
                # Optional: Gradient clipping to prevent exploding gradients
                nn.utils.clip_grad_norm_(model.parameters1(),max_norm=1.0)
                
                optimizer.step()
                
                # Detach hidden state to prevent backprop across batches
                hidden = hidden.detach()
                
                total_train_loss += loss.item()
            
            avg_train_loss = total_train_loss/len(trainx_loader)
            
            train_losses.append(avg_train_loss)
            
            
            #validation loop
            model.eval()
            total_val_loss = 0
            hidden = None
            
            with torch.no_grad():
                for x_val,y_val in zip(validx_loader,validy_loader):
                    x_val = x_val.to(device)
                    y_val = y_val.to(device)
                    
                    outputs, hidden = model(x_val,hidden)
                    outputs = outputs.squeeze(-1)
                    val_loss = criterion(outputs,y_val)
                    total_val_loss += val_loss.item()
                    
                    hidden = hidden.detach()
            
            avg_val_loss = total_val_loss/len(validx_loader)
            val_losses.append(avg_val_loss)
            
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'Training Loss: {avg_train_loss:.4f}')
            print(f'Validation Loss: {avg_val_loss:.4}')
            
        return train_losses, val_losses   

In [22]:
model = SimpleRNN(input_size=3,hidden_size=4,output_size=1)

train_model(model,x_train,y_train,x_valid,y_valid,10,0.001)

TypeError: 'int' object is not iterable